<a href="https://colab.research.google.com/github/doodger/dataScience/blob/master/Credit_card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Work in Progress. In this notebook I explored an imbalanced data set, the fraudulent credit card transaction set, for practice purposes.

Ideas and concepts being practiced include imbalanced data sets, weight adjustments, metrics of performance for imbalanced data sets, SVMs

In [36]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
!pip install xgboost
from xgboost import XGBClassifier #creates annoying warnings
pip install -U imbalanced-learn
from 


In [0]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1V97Aysa1jrTjQGkB7UD83jBfs0xzl2s_"})   # replace the id with id of file you want to access
downloaded.GetContentFile('creditcard.csv')        # replace the file name with your file
data = pd.read_csv('creditcard.csv')


The 28 features are PCA transformed from credit cards, for anonymity reasons. The amount is the quantity involved in the transaction. Time is the amount of time elapsed since the first transaction in the data set.

Feature class is the type of transaction: 0 for regular, 1 for fraudulent. 0.172% of all transactions are fraudulent.

In [16]:
data
#data.isna().sum()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [21]:
X = data.loc[0:284806,"Time":"Amount"]
y = data.loc[0:284806,"Class"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=data[["Class"]]) #stratified so the same proportion of 


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
6979,9100.0,1.031695,-0.239100,1.263742,0.706673,-0.596569,0.881408,-0.926847,0.387035,1.835033,-0.495540,2.483190,-1.379883,1.454231,1.443833,-0.271527,-0.064160,0.743789,-0.267440,-0.786819,-0.154745,-0.088669,0.103831,0.086787,-0.302488,-0.009499,0.418693,0.001717,0.005897,28.00
75678,56197.0,0.945842,-0.400033,-0.554717,1.778090,1.832547,4.439277,-0.716450,1.068586,0.011658,0.428787,-0.997941,0.115068,-0.224861,-0.299218,-1.192896,0.317419,-0.486874,-0.097167,-0.142326,0.189343,-0.111339,-0.472340,-0.201955,1.025456,0.688154,0.063892,0.006502,0.038923,127.78
273177,165461.0,1.972796,0.023564,-2.284224,0.314920,0.801832,-0.645918,0.364287,-0.319633,0.491260,-0.580251,-0.864060,0.198784,0.985330,-1.039925,0.926141,0.453112,0.134658,0.739730,-0.183527,0.059981,0.190571,0.558707,-0.156040,-0.015333,0.372213,-0.102059,-0.016075,-0.014329,79.75
92243,63844.0,-0.724890,1.431812,1.070661,0.177873,-0.389137,-1.177341,0.480596,0.268554,-0.751703,-0.778832,0.113763,0.750433,1.187114,-0.303875,0.715182,0.362324,0.233736,-0.331768,-0.229426,0.092721,-0.179470,-0.554079,0.101504,0.686850,-0.138130,0.050110,0.122756,0.042145,17.99
254837,156934.0,2.233813,-1.404392,-0.890508,-1.675541,-1.106820,-0.211679,-1.217358,-0.029322,-1.479710,1.708674,0.799486,0.063973,0.624345,-0.310915,-0.825310,-0.417823,0.277602,0.265495,0.072755,-0.377565,-0.133094,0.073654,0.286298,0.687941,-0.262333,-0.253437,0.010552,-0.049698,22.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20055,30755.0,-0.971882,0.791218,1.188455,0.476671,0.550972,-0.730329,0.800552,-0.167774,-0.709313,0.009847,-0.781492,-0.381871,0.110298,0.286046,1.279495,-0.355173,-0.064595,-0.025683,1.258731,0.264504,-0.068396,-0.086692,-0.169206,-0.075680,0.529698,0.591558,0.133994,0.142621,27.68
41983,40906.0,1.101534,0.246476,1.151972,2.582356,-0.682138,-0.011828,-0.421358,0.252161,-0.279030,0.798097,0.774648,-0.107846,-1.835828,0.445922,-0.611181,0.768743,-0.531960,0.382910,-0.661308,-0.271060,0.006283,-0.006944,0.020104,0.482631,0.329828,-0.011980,0.001763,0.015483,0.76
65341,51538.0,1.165504,-0.217335,0.108101,-0.039469,-0.452742,-0.605693,-0.066156,-0.061556,0.017205,0.043461,1.030236,0.440597,-0.476200,0.536954,0.440452,0.556831,-0.645099,0.295136,0.354094,0.030789,0.074729,0.062269,-0.143971,0.077963,0.356801,1.069177,-0.103301,-0.003741,60.90
32405,36787.0,0.906842,-0.137392,0.365476,1.287609,-0.509910,-0.616961,0.217670,-0.101436,-0.303871,0.169135,1.464058,0.931072,-0.053845,0.567718,0.239101,0.304994,-0.645577,0.387847,-0.306607,0.147646,0.220002,0.329221,-0.219393,0.559265,0.565357,-0.326028,-0.011716,0.038338,144.00


In [48]:
print("percentage of fraudulent transactions in train set:",len(y_train[y_train == 1])/len(y_train))
print("percentage of fraudulent transactions in test set:",len(y_test[y_test == 1])/len(y_test))
print("Ratio of negative to positive:", len(y_train[y_train==0]/len(y_train[y_train==1])))

percentage of fraudulent transactions in train set: 0.001729245759178389
percentage of fraudulent transactions in test set: 0.0017204452090867595
Ratio of negative to positive: 227451


Training on unmodified data with decision trees and xgboost classifier

In [44]:
model1 = DecisionTreeClassifier()
model1.fit(X_train,y_train)
y_pred1 = model1.predict(X_test)


model2 = XGBClassifier()
model2.fit(X_train,y_train, early_stopping_rounds = 10, eval_set = [(X_test,y_test)], eval_metric = "error", verbose = True)
y_pred2 = model2.predict(X_test)

[0]	validation_0-error:0.000966
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.000913
[2]	validation_0-error:0.000913
[3]	validation_0-error:0.000913
[4]	validation_0-error:0.000878
[5]	validation_0-error:0.000878
[6]	validation_0-error:0.000878
[7]	validation_0-error:0.00086
[8]	validation_0-error:0.00086
[9]	validation_0-error:0.00086
[10]	validation_0-error:0.00086
[11]	validation_0-error:0.00086
[12]	validation_0-error:0.00086
[13]	validation_0-error:0.00086
[14]	validation_0-error:0.000878
[15]	validation_0-error:0.000878
[16]	validation_0-error:0.00086
[17]	validation_0-error:0.00086
Stopping. Best iteration:
[7]	validation_0-error:0.00086



In [46]:
print("Accuracy score for decision tree:",accuracy_score(y_test,y_pred1))
print("Accuracy balanced score for decision tree:",balanced_accuracy_score(y_test,y_pred1))
cm = confusion_matrix(y_test,y_pred1,normalize='true')
cm_df = pd.DataFrame(cm)
cm_df.style.background_gradient(cmap='PiYG', axis=None)


Accuracy score for decision tree: 0.9989993328885924
Accuracy balanced score for decision tree: 0.8314216232356758


,0,1
0,0.999578,0.00042206
1,0.336735,0.663265


In [47]:

print("Accuracy score for xgboost:",accuracy_score(y_test,y_pred2))
print("Accuracy balanced score for xgboost:",balanced_accuracy_score(y_test,y_pred2))
cm = confusion_matrix(y_test,y_pred2,normalize='true')
cm_df = pd.DataFrame(cm)
cm_df.style.background_gradient(cmap='PiYG', axis=None)

Accuracy score for xgboost: 0.9991397773954567
Accuracy balanced score for xgboost: 0.811118974883144


,0,1
0,0.999789,0.00021103
1,0.377551,0.622449


In [49]:
model3 = XGBClassifier(scale_pos_weight=227451) #changing the scale
model3.fit(X_train,y_train, early_stopping_rounds = 10, eval_set = [(X_test,y_test)], eval_metric = "error", verbose = True) 
y_pred3 = model3.predict(X_test)

[0]	validation_0-error:0.775745
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.751729
[2]	validation_0-error:0.775148
[3]	validation_0-error:0.775148
[4]	validation_0-error:0.775657
[5]	validation_0-error:0.775148
[6]	validation_0-error:0.771848
[7]	validation_0-error:0.767547
[8]	validation_0-error:0.767547
[9]	validation_0-error:0.583705
[10]	validation_0-error:0.596099
[11]	validation_0-error:0.590692
[12]	validation_0-error:0.592026
[13]	validation_0-error:0.583248
[14]	validation_0-error:0.574049
[15]	validation_0-error:0.560602
[16]	validation_0-error:0.508637
[17]	validation_0-error:0.4944
[18]	validation_0-error:0.476177
[19]	validation_0-error:0.469541
[20]	validation_0-error:0.467873
[21]	validation_0-error:0.397511
[22]	validation_0-error:0.393403
[23]	validation_0-error:0.386995
[24]	validation_0-error:0.385766
[25]	validation_0-error:0.380236
[26]	validation_0-error:0.375935
[27]	validation_0-error:0.335329
[28]	validation_0-erro

In [50]:
print("Accuracy score for xgboost:",accuracy_score(y_test,y_pred3))
print("Accuracy balanced score for xgboost:",balanced_accuracy_score(y_test,y_pred3))
cm = confusion_matrix(y_test,y_pred3,normalize='true')
cm_df = pd.DataFrame(cm)
cm_df.style.background_gradient(cmap='PiYG', axis=None)

Accuracy score for xgboost: 0.9089217372985499
Accuracy balanced score for xgboost: 0.8881701632538215


,0,1
0,0.908993,0.0910066
1,0.132653,0.867347


In [51]:
model4 = XGBClassifier(scale_pos_weight=np.sqrt(227451)) #changing the scale; using sqrt tip to "limit multiplication of positive example"?
model4.fit(X_train,y_train, early_stopping_rounds = 10, eval_set = [(X_test,y_test)], eval_metric = "error", verbose = True) 
y_pred4 = model4.predict(X_test)

[0]	validation_0-error:0.026351
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.026351
[2]	validation_0-error:0.026456
[3]	validation_0-error:0.010832
[4]	validation_0-error:0.015554
[5]	validation_0-error:0.024192
[6]	validation_0-error:0.013132
[7]	validation_0-error:0.015168
[8]	validation_0-error:0.014431
[9]	validation_0-error:0.008778
[10]	validation_0-error:0.007602
[11]	validation_0-error:0.007549
[12]	validation_0-error:0.008058
[13]	validation_0-error:0.007619
[14]	validation_0-error:0.007549
[15]	validation_0-error:0.008514
[16]	validation_0-error:0.008356
[17]	validation_0-error:0.008479
[18]	validation_0-error:0.008356
[19]	validation_0-error:0.008637
[20]	validation_0-error:0.008778
[21]	validation_0-error:0.00862
Stopping. Best iteration:
[11]	validation_0-error:0.007549



In [52]:
print("Accuracy score for xgboost:",accuracy_score(y_test,y_pred4))
print("Accuracy balanced score for xgboost:",balanced_accuracy_score(y_test,y_pred4))
cm = confusion_matrix(y_test,y_pred4,normalize='true')
cm_df = pd.DataFrame(cm)
cm_df.style.background_gradient(cmap='PiYG', axis=None)

Accuracy score for xgboost: 0.992451107756048
Accuracy balanced score for xgboost: 0.9096338345411321


,0,1
0,0.992737,0.00726294
1,0.173469,0.826531
